In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 17.6 MB/s eta 0:00:00


In [3]:
try:
  from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op  # pylint:disable=g-import-not-at-top
except ImportError:
  frontend_op = None

In [4]:
from typing import Tuple, Optional

import tensorflow as tf
from tensorflow import keras
import tensorflow_io as tfio

import numpy as np
import pandas as pd

import os
import sys
import tarfile
import hashlib
import re
import glob

import random
import math

import IPython.display as ipd
from tensorflow.python.util import compat

In [5]:
LIB_PATH = '/content/drive/MyDrive/GSC/GSC_helper'
sys.path.append(LIB_PATH)
from utils import _download, unzipzip, zipzip
from GSC import download_GSC
from GSC12 import SpeechCommands12

## GSC

In [6]:
ZIP_MAP = download_GSC('https://drive.google.com/file/d/1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-6St6zZqFvDy0JHhHTNV9UORH0mTjoDp/view?usp=drive_link',
                       '/content/GSC_8',
                       end = '.zip')
CSV_MAP = download_GSC('https://drive.google.com/file/d/1-GQ55fRsP1zNat93Yoc2tM7tUqUPiuch/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-FIGYCKCiKdTqZjcdbqVdKTUBt6-R1w9/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-Mpiyo-AE5at9ahnO8qD9ljl75lr7DJA/view?usp=drive_link',
                       '/content/GSC_8',
                       end = '.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf
From (redirected): https://drive.google.com/uc?id=1nvubIeIQ4K7cte-7JRIKpaXJ0rW4FoCf&confirm=t&uuid=a886592b-746e-449c-a5fe-b7aeff8d1650
To: /content/GSC_8/train.zip
100%|██████████| 58.6M/58.6M [00:01<00:00, 34.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5Bglq0ihfzZ4tM_zZ6dxAsnoYv1YzAm
To: /content/GSC_8/val.zip
100%|██████████| 5.66M/5.66M [00:00<00:00, 38.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-6St6zZqFvDy0JHhHTNV9UORH0mTjoDp
To: /content/GSC_8/test.zip
100%|██████████| 6.34M/6.34M [00:00<00:00, 18.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-GQ55fRsP1zNat93Yoc2tM7tUqUPiuch
To: /content/GSC_8/train.csv
100%|██████████| 600k/600k [00:00<00:00, 7.48MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-FIGYCKCiKdTqZjcdbqVdKTUBt6-R1w9
To: /content/GSC_8/val.csv
100%|██████████| 59.0k/59.0k [00:00<00:00, 3.78MB/s]
Downloading...
From:

In [7]:
import pandas as pd
import os

class GSC_8(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 root: str,
                 zip_map: dict,
                 csv_map: dict,
                 unzip: bool = True,
                 subset: str = 'train',
                 batch_size: int = 32,
                 shuffle: bool = True):
        'Initialization'
        super().__init__()
        local_path = os.path.join(root, subset)
        self.root = root
        if not os.path.exists(local_path):
            os.mkdir(local_path)
            unzipzip(zip_map[subset], local_path)
        if unzip:
            unzipzip(zip_map[subset], local_path)
        self.csv = pd.read_csv(csv_map[subset])
        self.subset = subset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the numbber of batches per epoch'
        return int(np.floor(len(self.csv)/self.batch_size))

    def __getitem__(self, index):
        'Generate on batch of data'
        # Generate indexes of the batcch
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Undates indexes after each epoch'
        self.indexes = np.arange(len(self.csv))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X: (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i in indexes:
            # Store sample
            row = self.csv.iloc[i]
            X.append(np.load(os.path.join(self.root, row['link']))['arr_0'])

            # Store class
            y.append(row['label'])
        X = np.stack(X, axis = 0)
        y = np.stack(y, axis = 0)
        return tf.convert_to_tensor(X), tf.convert_to_tensor(y)

In [8]:
train_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'train', batch_size = 128, shuffle = True)
val_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'val', batch_size = 128, shuffle = False)
test_dataloader = GSC_8('/content/GSC_8', ZIP_MAP, CSV_MAP, unzip = False, subset = 'test', batch_size = 10, shuffle = False)

Extracted /content/GSC_8/train.zip
Extracted /content/GSC_8/val.zip
Extracted /content/GSC_8/test.zip


In [ ]:
X, y = next(iter(train_dataloader))
X.shape

TensorShape([128, 49, 40, 1])

In [15]:
model = keras.models.Sequential([
    keras.layers.Input(shape = (49, 40, 1)),
    keras.layers.Conv2D(3, kernel_size = 5, strides = 1, padding = 'same', activation = 'relu', use_bias = False),
    keras.layers.Conv2D(3, kernel_size = 3, strides = 1, padding = 'same', groups = 3, use_bias = False),
    keras.layers.Conv2D(16, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(16, kernel_size = 3, strides = 1, padding = 'same', groups = 16, use_bias = False),
    keras.layers.Conv2D(32, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(32, kernel_size = 3, strides = 1, padding = 'same', groups = 32, use_bias = False),
    keras.layers.Conv2D(32, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(32, kernel_size = 3, strides = 1, padding = 'same', groups = 32, use_bias = False),
    keras.layers.Conv2D(64, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(64, kernel_size = 3, strides = 1, padding = 'same', groups = 64, use_bias = False),
    keras.layers.Conv2D(64, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(64, kernel_size = 3, strides = 2, padding = 'same', groups = 64, use_bias = False),
    keras.layers.Conv2D(128, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(keepdims = True),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(8),
    keras.layers.Softmax()])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 49, 40, 3)         75        
                                                                 
 conv2d_14 (Conv2D)          (None, 49, 40, 3)         27        
                                                                 
 conv2d_15 (Conv2D)          (None, 49, 40, 16)        48        
                                                                 
 batch_normalization_6 (Bat  (None, 49, 40, 16)        64        
 chNormalization)                                                
                                                                 
 re_lu_6 (ReLU)              (None, 49, 40, 16)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 24, 20, 16)        0         
 g2D)                                                 

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0005, weight_decay = 0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [17]:
model_ckpt = keras.callbacks.ModelCheckpoint('/content/bkup_model_ckpt',
                                             monitor = 'val_accuracy',
                                             mode = 'max',
                                             save_best_only = True,
                                             )

In [18]:
EPOCHS = 50
history = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=EPOCHS,
    callbacks = [tf.keras.callbacks.EarlyStopping(verbose=1, patience=4),
                 model_ckpt],
)

Epoch 1/50
170/170 [==============================] - 25s 120ms/step - loss: 1.7951 - accuracy: 0.3082 - val_loss: 2.1013 - val_accuracy: 0.1332
Epoch 2/50
170/170 [==============================] - 21s 121ms/step - loss: 1.2407 - accuracy: 0.5689 - val_loss: 2.2135 - val_accuracy: 0.1422
Epoch 3/50
170/170 [==============================] - 19s 113ms/step - loss: 0.9187 - accuracy: 0.6926 - val_loss: 1.0732 - val_accuracy: 0.6086
Epoch 4/50
170/170 [==============================] - 21s 122ms/step - loss: 0.7692 - accuracy: 0.7412 - val_loss: 0.5887 - val_accuracy: 0.7988
Epoch 5/50
170/170 [==============================] - 19s 114ms/step - loss: 0.6927 - accuracy: 0.7664 - val_loss: 0.5910 - val_accuracy: 0.8102
Epoch 6/50
170/170 [==============================] - 24s 140ms/step - loss: 0.6348 - accuracy: 0.7847 - val_loss: 0.4477 - val_accuracy: 0.8492
Epoch 7/50
170/170 [==============================] - 17s 103ms/step - loss: 0.5942 - accuracy: 0.7994 - val_loss: 0.5360 - val_ac

In [31]:
best_model = keras.models.load_model('/content/bkup_model_ckpt')
best_model.evaluate(test_dataloader)

292/292 [==============================] - 4s 10ms/step - loss: 0.3789 - accuracy: 0.8757


[0.3789340853691101, 0.875684916973114]

In [20]:
_, baseline_model_accuracy = model.evaluate(test_dataloader, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.8654109835624695


NameError: name 'tempfile' is not defined

In [21]:
keras_file = '/content/bkup_model_ckpt'

## Pruning

In [22]:
!pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.0 MB/s eta 0:00:00


In [23]:
import tempfile
import tensorflow_model_optimization as tfmot

In [32]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after after 5 epochs.
end_epoch = 10

num_iterations_per_epoch = len(train_dataloader)
end_step =  num_iterations_per_epoch * end_epoch

# Define parameters for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.5,
                                                               final_sparsity=0.75,
                                                               begin_step=0,
                                                               end_step=end_step),
#      'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
}

# Try to apply pruning wrapper with pruning policy parameter.
try:
  model_for_pruning = prune_low_magnitude(best_model, **pruning_params)
except ValueError as e:
  print(e)

In [33]:
model_for_pruning.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-4),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 49, 40, 3)         152       
 _13 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d  (None, 49, 40, 3)         56        
 _14 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d  (None, 49, 40, 16)        98        
 _15 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_batch_  (None, 49, 40, 16)        65        
 normalization_6 (PruneLowM                                      
 agnitude)                                                       
                                                      

In [34]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_dataloader, validation_data = val_dataloader,
                  callbacks=callbacks, epochs = end_epoch)

Epoch 1/10
170/170 [==============================] - 32s 113ms/step - loss: 0.8907 - accuracy: 0.7029 - val_loss: 3.8155 - val_accuracy: 0.2543
Epoch 2/10
170/170 [==============================] - 20s 116ms/step - loss: 1.2008 - accuracy: 0.5809 - val_loss: 2.0004 - val_accuracy: 0.3902
Epoch 3/10
170/170 [==============================] - 19s 110ms/step - loss: 1.1618 - accuracy: 0.5968 - val_loss: 2.1526 - val_accuracy: 0.2859
Epoch 4/10
170/170 [==============================] - 18s 107ms/step - loss: 1.1548 - accuracy: 0.6007 - val_loss: 1.0252 - val_accuracy: 0.6500
Epoch 5/10
170/170 [==============================] - 19s 114ms/step - loss: 1.1483 - accuracy: 0.6103 - val_loss: 1.0043 - val_accuracy: 0.6277
Epoch 6/10
170/170 [==============================] - 20s 115ms/step - loss: 1.0302 - accuracy: 0.6556 - val_loss: 1.3251 - val_accuracy: 0.4922
Epoch 7/10
170/170 [==============================] - 18s 108ms/step - loss: 0.9730 - accuracy: 0.6811 - val_loss: 0.7353 - val_ac

In [35]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

<ipython-input-35-395ce8c5b5c5>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: /tmp/tmpcewrykx5.h5


In [36]:
model_for_export.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 49, 40, 3)         75        
                                                                 
 conv2d_14 (Conv2D)          (None, 49, 40, 3)         27        
                                                                 
 conv2d_15 (Conv2D)          (None, 49, 40, 16)        48        
                                                                 
 batch_normalization_6 (Bat  (None, 49, 40, 16)        64        
 chNormalization)                                                
                                                                 
 re_lu_6 (ReLU)              (None, 49, 40, 16)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 24, 20, 16)        0         
 g2D)                                                 

## Clustering

In [ ]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_conv2d_38 (Cluster  (None, 49, 40, 3)         166       
 Weights)                                                        
                                                                 
 cluster_conv2d_39 (Cluster  (None, 49, 40, 3)         70        
 Weights)                                                        
                                                                 
 cluster_conv2d_40 (Cluster  (None, 49, 40, 16)        112       
 Weights)                                                        
                                                                 
 cluster_batch_normalizatio  (None, 49, 40, 16)        64        
 n_18 (ClusterWeights)                                           
                                                                 
 cluster_re_lu_18 (ClusterW  (None, 49, 40, 16)       

In [ ]:
clustered_model.fit(
  train_dataloader,
  validation_data = val_dataloader,
  epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


170/170 [==============================] - 26s 108ms/step - loss: 0.5634 - accuracy: 0.8120 - val_loss: 0.4252 - val_accuracy: 0.8664
Epoch 2/5
170/170 [==============================] - 18s 103ms/step - loss: 0.4410 - accuracy: 0.8491 - val_loss: 0.3868 - val_accuracy: 0.8793
Epoch 3/5
170/170 [==============================] - 17s 102ms/step - loss: 0.4126 - accuracy: 0.8577 - val_loss: 0.3767 - val_accuracy: 0.8805
Epoch 4/5
170/170 [==============================] - 17s 100ms/step - loss: 0.4006 - accuracy: 0.8613 - val_loss: 0.3634 - val_accuracy: 0.8805
Epoch 5/5
170/170 [==============================] - 25s 148ms/step - loss: 0.3900 - accuracy: 0.8658 - val_loss: 0.3591 - val_accuracy: 0.8852


In [ ]:
_, clustered_model_accuracy = clustered_model.evaluate(test_dataloader, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.8606164455413818
Clustered test accuracy: 0.8732876777648926


In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
keras.models.save_model(final_model, clustered_keras_file,
                           include_optimizer=False)

<ipython-input-60-9613704959c8>:5: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(final_model, clustered_keras_file,


Saving clustered model to:  /tmp/tmp2gvj901h.h5


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.GlorotUniform'>, which may lead to improper serialization.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.Zeros'>, which may lead to improper serialization.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.Ones'>, which

## 2TFLite

In [37]:
from tqdm import tqdm

test_specs = []
test_labels = []

for i in tqdm(range(len(test_dataloader.csv))):
    row = test_dataloader.csv.iloc[i]
    spec = np.load(os.path.join('/content/GSC_8', row['link']))['arr_0']
    test_specs.append(spec)
    test_labels.append(row['label'])

test_specs = tf.convert_to_tensor(np.stack(test_specs, axis = 0))
test_labels = tf.convert_to_tensor(np.stack(test_labels))

100%|██████████| 2925/2925 [00:01<00:00, 1591.25it/s]


In [38]:
test_specs.shape

TensorShape([2925, 49, 40, 1])

In [39]:
test_labels.shape

TensorShape([2925])

In [40]:
base_model = keras.models.load_model(keras_file)

Baseline

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
tflite_model = converter.convert()

Post-train Quant Only

In [42]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(test_specs).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Pruned_Model

In [43]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant_prune = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Clusted_model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant_cluster = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [44]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/gsc_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the dynamic range quantized model:
tflite_model_quant_file = tflite_models_dir/"gsc_tflite_backup_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

51304

In [45]:
tflite_model_file = tflite_models_dir/"gsc_model_backup.tflite"
tflite_model_file.write_bytes(tflite_model)

116304

In [ ]:
tflite_model_quant_cluster_file = tflite_models_dir/"gsc_tflite_backup_quant_cluster.tflite"
tflite_model_quant_cluster_file.write_bytes(tflite_model_quant_cluster)

51368

In [46]:
tflite_model_quant_prune_file = tflite_models_dir/"gsc_tflite_backup_quant_prune.tflite"
tflite_model_quant_prune_file.write_bytes(tflite_model_quant_prune)

51304

In [47]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
    global test_specs

    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path = str(tflite_file))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    predictions = np.zeros((len(test_image_indices), ), dtype = int)
    for i, test_image_index in enumerate(test_image_indices):
        test_image = test_specs[test_image_index]

        # Check if the input type is quantized, the rescale input data to to uint8
        if input_details['dtype'] == np.int8:
            input_scale, input_zero_point = input_details['quantization']
            test_image = test_image/input_scale + input_zero_point

        test_image = np.expand_dims(test_image, axis = 0).astype(input_details['dtype'])
        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]

        predictions[i] = output.argmax()

    return predictions

# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_specs
  global test_labels

  test_image_indices = range(test_specs.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_specs)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_specs)))

In [48]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Quantized model accuracy is 87.3504% (Number of test samples=2925)


In [ ]:
evaluate_model(tflite_model_quant_cluster_file, model_type="Quantized")

Quantized model accuracy is 87.2821% (Number of test samples=2925)


In [49]:
evaluate_model(tflite_model_quant_prune_file, model_type="Quantized")

Quantized model accuracy is 77.2650% (Number of test samples=2925)


In [50]:
evaluate_model(tflite_model_file, model_type="Float32")

Float32 model accuracy is 87.5214% (Number of test samples=2925)


In [51]:
import shutil
shutil.copy2(str(tflite_model_quant_file), '/content')

'/content/gsc_tflite_backup_quant.tflite'

In [ ]:
shutil.copy2(str(tflite_model_quant_cluster_file), '/content')

'/content/gsc_tflite_backup_quant_cluster.tflite'

In [52]:
shutil.copy2(str(tflite_model_quant_prune_file), '/content')

'/content/gsc_tflite_backup_quant_prune.tflite'

In [ ]:
# Save the file as a C source file
!xxd -i /content/gsc_tflite_backup_quant.tflite > /content/bkup_model.cc
# Print the source file
!cat /content/bkup_model.cc

unsigned char _content_gsc_tflite_backup_quant_tflite[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x84, 0x00, 0x00, 0x00, 0xdc, 0x00, 0x00, 0x00,
  0x20, 0x74, 0x00, 0x00, 0x30, 0x74, 0x00, 0x00, 0xe0, 0xc7, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0xae, 0x8a, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x98, 0xff, 0xff, 0xff,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00,
  0x73, 0x6f, 0x66, 0x74, 0x6d, 0x61, 0x78, 0x5f, 0x33, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x

In [ ]:
# Save the file as a C source file
!xxd -i /content/gsc_tflite_backup_quant_cluster.tflite > /content/bkup_model_cluster.cc
# Print the source file
!cat /content/bkup_model_cluster.cc

unsigned char _content_gsc_tflite_backup_quant_cluster_tflite[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x84, 0x00, 0x00, 0x00, 0xdc, 0x00, 0x00, 0x00,
  0x20, 0x74, 0x00, 0x00, 0x30, 0x74, 0x00, 0x00, 0xe0, 0xc7, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0xae, 0x8a, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x98, 0xff, 0xff, 0xff,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00,
  0x73, 0x6f, 0x66, 0x74, 0x6d, 0x61, 0x78, 0x5f, 0x33, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 

In [53]:
# Save the file as a C source file
!xxd -i /content/gsc_tflite_backup_quant_prune.tflite > /content/bkup_model_prune.cc
# Print the source file
!cat /content/bkup_model_prune.cc

unsigned char _content_gsc_tflite_backup_quant_prune_tflite[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x84, 0x00, 0x00, 0x00, 0xdc, 0x00, 0x00, 0x00,
  0x20, 0x74, 0x00, 0x00, 0x30, 0x74, 0x00, 0x00, 0xa0, 0xc7, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0xae, 0x8a, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x98, 0xff, 0xff, 0xff,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00,
  0x73, 0x6f, 0x66, 0x74, 0x6d, 0x61, 0x78, 0x5f, 0x31, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x